In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Input, SimpleRNN
from tensorflow.keras.utils import pad_sequences # Import pad_sequences

# Loading dataset from tensorflow library
from tensorflow.keras.datasets import imdb

In [2]:
# Load the dataset
max_features = 10000 # vocabulary size
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)
print(f'Training data shape: {x_train.shape}, training labels shape: {y_train.shape}')
print(f'Testing data shape:{x_test.shape}, Testing Label Shape:{y_test.shape}')

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training data shape: (25000,), training labels shape: (25000,)
Testing data shape:(25000,), Testing Label Shape:(25000,)


In [3]:
# Inspect a Sample Review and its label
sample_review = x_train[0]
sample_label = y_train[0]
print(f'Review: {sample_review}')
print(f'Label: {sample_label}')

Review: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Label: 1


In [4]:
# Padding sequence

max_length = 500
x_train = sequence.pad_sequences(x_train, maxlen = max_length)
x_test = sequence.pad_sequences(x_test, maxlen = max_length)
x_train

# Pre padding is used bydefault

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

# Training Simple RNN

In [5]:
model = Sequential()
model.add(Input(shape = (max_length,)))
model.add(Embedding(max_features, 128, input_length = max_length)) # 128 = out dimentions
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 500, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Create an instance of Early Stopping Callbacks
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss',patience=5, restore_best_weights= True)
early_stopping

In [8]:
#compile the model
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [9]:
# Training the model with early stopping
history =  model.fit(
    x_train, y_train, epochs=10,batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 156s 247ms/step - accuracy: 0.5671 - loss: 7594937.5000 - val_accuracy: 0.5846 - val_loss: 0.6580
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 249ms/step - accuracy: 0.6742 - loss: 0.6263 - val_accuracy: 0.6350 - val_loss: 0.6274
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 205s 254ms/step - accuracy: 0.7311 - loss: 0.5592 - val_accuracy: 0.6650 - val_loss: 0.5987
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 154s 246ms/step - accuracy: 0.7847 - loss: 0.4748 - val_accuracy: 0.6988 - val_loss: 0.5645
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 206s 252ms/step - accuracy: 0.8405 - loss: 0.3771 - val_accuracy: 0.7248 - val_loss: 0.5957
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 154s 247ms/step - accuracy: 0.6035 - loss: nan - val_accuracy: 0.5062 - val_loss: nan
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 202s 247ms/step - accuracy: 0.4988 - loss: nan - val_accuracy: 0.5062 - val_loss: nan
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 158s 253ms/step - accuracy: 0.5006 - loss:

In [11]:
## Saving model in h5 file
model.save('simple_rnn_model.h5')